# Pre-requisites for Amsterdam UMC DB


In [2]:
# sets *your* project id
PROJECT_ID = "esicmdatathon2024" #@param {type:"string"}


In [3]:
# sets default dataset for AmsterdamUMCdb
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'version1_5_0' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

In [4]:
import os
from google.colab import auth

# all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

auth.authenticate_user()
print('Authenticated')


Authenticated


In [5]:
%load_ext google.colab.data_table
from google.colab.data_table import DataTable

# change default limits:
DataTable.max_columns = 50
DataTable.max_rows = 30000


In [6]:
from google.cloud.bigquery import magics
from google.cloud import bigquery

# sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config


In [28]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns
sns.set_style('darkgrid')

# Ventilator measurements for comparison with tracheostomy
I think, if the patient enters already ventilated the time to start vent date is near 0. If the patient is intubated during stay, then there is an intubation date.   

In [78]:
%%bigquery vent_pat --project esicmdatathon2024
with tracheal as (
select d.person_id, d.visit_occurrence_id
, min(device_exposure_start_datetime) as insert_trach
, timestamp_diff(min(device_exposure_start_datetime), visit_start_datetime, hour)/24.0 as days_admit_to_first_trach
  from device_exposure d
inner join concept c2 on d.device_concept_id= c2.concept_id
inner join visit_occurrence v on d.visit_occurrence_id = v.visit_occurrence_id
          and timestamp_diff(v.visit_end_datetime, v.visit_start_datetime, hour )/24.0 >= 2 -- filter for longer patients only
where NOT d.provider_id IS NULL -- ignore unvalidated device data
  and c2.concept_id in ( 45772284 -- Tracheostomy tube speech valve
                        ,4097216 -- Endotracheal tube
                        ,4044008 -- Tracheostomy tube
                        ,4208623 -- Double lumen tracheobronchial tube
        )
group by d.person_id, d.visit_occurrence_id, visit_start_datetime
),
vent_pat as (
select m.person_id, m.visit_occurrence_id
      , count(distinct m.measurement_date ) as days_vent_param
      , min(measurement_datetime) as first_date_vent
      , timestamp_diff(min(measurement_datetime), visit_start_datetime, HOUR)/24.0 as days_admit_to_first_vent
  from measurement m
inner join concept c2 on m.measurement_concept_id= c2.concept_id
inner join visit_occurrence v on m.visit_occurrence_id = v.visit_occurrence_id
          and timestamp_diff(v.visit_end_datetime, v.visit_start_datetime, hour )/24.0 >= 2 -- filter for longer patients only
where NOT m.provider_id IS NULL -- ignore unvalidated device data
  and c2.concept_id in ( 3004921 -- Ventilation mode Ventilator
                        ,3007646 -- Breath rate spontaneous and mechanical --on ventilator
                        ,3016078 -- Maximum [Pressure] Respiratory system airway opening --during inspiration on ventilator
                        ,21490855 -- PEEP Respiratory system --on ventilator
                        ,3025408 -- Oxygen/Inspired gas Respiratory system by O2 Analyzer --on ventilator
                        ,42527120 -- Expired minute Volume during Mechanical ventilation
                        ,3016166 --Tidal volume inspired spontaneous+mechanical Measured --on ventilator
        )
  and measurement_source_value not like '%Bipap%' -- ignore NIV
  and measurement_source_value not like '%Zephyros%' -- ignore NIV
  and measurement_source_value not like '%(Set)%' -- ignore settings
group by m.person_id, m.visit_occurrence_id, visit_start_datetime
)
select ifnull(v.person_id, t.person_id) as person_id
, ifnull(v.visit_occurrence_id, t.visit_occurrence_id) as visit_occurrence_id
, first_date_vent, days_admit_to_first_vent, days_vent_param
, insert_trach, days_admit_to_first_trach
from vent_pat v full outer join tracheal t
on v.visit_occurrence_id = t.visit_occurrence_id


Query is running:   0%|          |

Downloading:   0%|          |

In [77]:

vent_pat


,person_id,visit_occurrence_id,first_date_vent,days_admit_to_first_vent,days_vent_param,insert_trach,days_admit_to_first_trach
0,568,654,2006-01-02 23:30:00+00:00,0.958333,37,NaT,NaN
1,3545,4080,2006-01-01 02:45:00+00:00,0.083333,8,NaT,NaN
2,20314,23540,2013-01-01 00:24:00+00:00,0.000000,2,NaT,NaN
3,20156,23350,2013-03-05 22:37:00+00:00,0.041667,30,2013-03-28 16:52:00+00:00,22.791667
4,647,738,2013-01-01 00:09:00+00:00,0.000000,6,2013-01-11 19:41:00+00:00,10.791667
...,...,...,...,...,...,...,...
7103,2762,3194,2013-01-17 05:51:00+00:00,16.208333,2,NaT,NaN
7104,4973,5738,2013-01-01 00:00:00+00:00,0.000000,1,NaT,NaN
7105,14743,17073,2006-01-01 02:04:00+00:00,0.083333,1,NaT,NaN
7106,4074,4688,2006-01-01 16:03:00+00:00,0.666667,2,NaT,NaN
